<a href="https://colab.research.google.com/github/anscch/ATPAD-LAIDEA-UNAM/blob/main/POLLUTION_ATPAD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***POLLUTION_ATPAD***


# DATASET CONSTRUCTION

Parquet files contains all data from each station. This files will need to be updated as new inforatin is available. To update such files.....

First cleaning steps are carried out by concatenating all station's files into a single sataset since all parameters are the same between station. For further analysis such as outlier identification, data must be splited by region.

In [42]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import duckdb
connection = duckdb.connect()
import os
import datetime

mor_data_raw = pd.read_parquet('/content/drive/MyDrive/ATPAD_COLAB/PARQUET_FILES/RUOA_MORE_1h_CAire_2024.parquet').reset_index(drop=True)
sal_data_raw = pd.read_parquet('/content/drive/MyDrive/ATPAD_COLAB/PARQUET_FILES/RUOA_SLLO_1h_CAire_2024.parquet').reset_index(drop=True)
agu_data_raw = pd.read_parquet('/content/drive/MyDrive/ATPAD_COLAB/PARQUET_FILES/RUOA_AGSC_1h_CAire_2024.parquet').reset_index(drop=True)


#filling gaps in timestamp

dfs = [mor_data_raw, sal_data_raw, agu_data_raw, ]

for i in range(len(dfs)):
    full_range = pd.date_range(start=dfs[i]['Time'].min(), end=dfs[i]['Time'].max(), freq='h')
    dfs[i]= dfs[i].set_index('Time')
    dfs[i]= dfs[i].reindex(full_range)

dfs[0]['region'] = 'Morelia'
dfs[1]['region'] = 'Saltillo'
dfs[2]['region'] = 'Aguascalientes'

pollution_raw = pd.concat(dfs).reset_index(drop=False)
pollution_raw = pollution_raw.rename(columns={'index': 'Time'})

pollution_raw

,Time,O3,SO2,NO2,NO,CO,PM10,PM25,region
0,2015-08-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Morelia
1,2015-08-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Morelia
2,2015-08-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Morelia
3,2015-08-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Morelia
4,2015-08-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Morelia
...,...,...,...,...,...,...,...,...,...
197407,2018-10-15 23:00:00,9.85,0.54,9.86,7.37,NaN,9.32,NaN,Aguascalientes
197408,2018-10-16 00:00:00,7.70,0.53,9.75,7.33,NaN,10.16,NaN,Aguascalientes
197409,2018-10-16 01:00:00,9.55,0.51,9.06,7.30,NaN,10.60,NaN,Aguascalientes
197410,2018-10-16 02:00:00,10.32,0.55,8.77,7.32,NaN,-1.63,NaN,Aguascalientes


# CLEANING DATASET
In this section, cleaning and validation criteria are appied to the entire dataset.


SETTING AS NULL VALUES THOSE BELOW NEGATIVE DETECTION LIMIT

Data between the negative and positive detection limit values are replaced by half the detection limit value.

In [43]:
s = '''
select
    time,

    CASE when
        (
            (O3 > -0.03) or (O3 is NULL)
        )
        then O3 else NULL
        END as O3,

    CASE when
        (
            (SO2 > -0.5) or (SO2 is NULL)
        )
        then SO2 else NULL
        END as SO2,

    CASE when
        (
            (NO2 > -0.4) or (NO2 is NULL)
        )
        then NO2 else NULL
        END as NO2,

    CASE when
        (
            (NO > -0.4) or (NO is NULL)
        )
        then NO else NULL
        END as NO,

    CASE when
        (
            (CO > -0.04) or (CO is NULL)
        )
        then CO else NULL
        END as CO,

    CASE when
        (
            (PM10 > -4.0) or (PM10 is NULL)
        )
        then PM10 else NULL
        END as PM10,

    CASE when
        (
            (PM25 > -4.0) or (PM25 is NULL)
        )
        then PM25 else NULL
        END as PM25,

    region

from pollution_raw
'''
pollution_clean = connection.execute(s).df()

SETTING VALUES BETWEEN NEGATIVE AND POSITIVE DETECTION LIMIT VALUES AS HALF OF THE DETECTION LIMIT VALUE

In [44]:
s = '''
select
    time,

    CASE when
        (
            (O3 > 0.03) or (O3 is NULL)
        )
        then O3 else 0.015
        END as O3,

    CASE when
        (
            (SO2 > 0.5) or (SO2 is NULL)
        )
        then SO2 else 0.25
        END as SO2,

    CASE when
        (
            (NO2 > 0.4) or (NO2 is NULL)
        )
        then NO2 else 0.2
        END as NO2,

    CASE when
        (
            (NO > 0.4) or (NO is NULL)
        )
        then NO else 0.2
        END as NO,

    CASE when
        (
            (CO > 0.04) or (CO is NULL)
        )
        then CO else 0.02
        END as CO,

    CASE when
        (
            (PM10 > 4.0) or (PM10 is NULL)
        )
        then PM10 else 2.0
        END as PM10,

    CASE when
        (
            (PM25 > 4.0) or (PM25 is NULL)
        )
        then PM25 else 2.0
        END as PM25,

    region

from pollution_clean
'''
pollution_clean = connection.execute(s).df()

REMOVING EXTRAORDINARY HIGH VALUES ABOVE SPAN LIMITS

In [45]:
s = '''
select
    time,

    CASE when
        (
            (O3 <= 500.0) or (O3 is NULL)
        )
        then O3 else NULL
        END as O3,

    CASE when
        (
            (SO2 <= 1000.0) or (SO2 is NULL)
        )
        then SO2 else NULL
        END as SO2,

    CASE when
        (
            (NO2 <= 300.0) or (NO2 is NULL)
        )
        then NO2 else NULL
        END as NO2,

    CASE when
        (
            (NO <= 500.0) or (NO is NULL)
        )
        then NO else NULL
        END as NO,

    CASE when
        (
            (CO <= 25.0) or (CO is NULL)
        )
        then CO else NULL
        END as CO,

    CASE when
        (
            (PM10 <= 300.0) or (PM10 is NULL)
        )
        then PM10 else NULL
        END as PM10,

    CASE when
        (
            (PM25 <= 300.0) or (PM25 is NULL)
        )
        then PM25 else NULL
        END as PM25,

    region

from pollution_clean
'''
pollution_clean = connection.execute(s).df()

#Z-SCORE EVALUATION FOR OUTLIER DETECTION

 Z-score thresholds can be modified as necesary. Default values were defined for long data series (more than five years). Thresholds may drecrease as the specific period does.

In [46]:
dft = pollution_clean.groupby('region').agg(o3_avg=('O3','mean'),
                            o3_std=('O3', 'std'),
                            so2_avg=('SO2','mean'),
                            so2_std=('SO2', 'std'),
                            no2_avg=('NO2','mean'),
                            no2_std=('NO2', 'std'),
                            no_avg=('NO','mean'),
                            no_std=('NO', 'std'),
                            co_avg=('CO','mean'),
                            co_std=('CO', 'std'),
                            pm10_avg=('PM10','mean'),
                            pm10_std=('PM10', 'std'),
                            pm25_avg=('PM25','mean'),
                            pm25_std=('PM25', 'std'),
                            ).reset_index()

pollution_clean = connection.execute('''
with qz as (select *,
    (pollution_clean.O3 - dft.o3_avg)/dft.o3_std as o3_zvalue,
    (pollution_clean.SO2 - dft.so2_avg)/dft.so2_std as so2_zvalue,
    (pollution_clean.NO2 - dft.no2_avg)/dft.no2_std as no2_zvalue,
    (pollution_clean.NO - dft.no_avg)/dft.no_std as no_zvalue,
    (pollution_clean.CO - dft.co_avg)/dft.co_std as co_zvalue,
    (pollution_clean.PM10 - dft.pm10_avg)/dft.pm10_std as pm10_zvalue,
    (pollution_clean.PM25 - dft.pm25_avg)/dft.pm25_std as pm25_zvalue,


    from pollution_clean

    left join dft on dft.region = pollution_clean.region)

select
    Time,
    region,
    CASE when
        (
            (o3_zvalue between -4 and 4) or (O3 is NULL)
        )
        then O3 else NULL
    END as O3,

    CASE when
        (
            (so2_zvalue between -4 and 4) or (SO2 is NULL)
        )
        then SO2 else NULL
    END as SO2,

    CASE when
        (
            (no2_zvalue between -6 and 6) or (NO2 is NULL)
        )
        then NO2 else NULL
    END as NO2,

    CASE when
        (
            (no_zvalue between -4 and 4) or (NO is NULL)
        )
        then NO else NULL
    END as NO,

    CASE when
        (
            (co_zvalue between -4 and 4) or (CO is NULL)
        )
        then CO else NULL
    END as CO,

    CASE when
        (
            (pm10_zvalue between -6 and 6) or (PM10 is NULL)
        )
        then PM10 else NULL
    END as PM10,

    CASE when
        (
            (pm25_zvalue between -6 and 6) or (PM25 is NULL)
        )
        then PM25 else NULL
    END as PM25



from qz
''').df()

pollution_clean.describe()

pollution_clean

,Time,region,O3,SO2,NO2,NO,CO,PM10,PM25
0,2017-10-24 12:00:00,Saltillo,NaN,NaN,4.45,1.49,0.81,37.45,NaN
1,2017-10-24 13:00:00,Saltillo,NaN,NaN,3.54,1.51,0.80,25.24,NaN
2,2017-10-24 14:00:00,Saltillo,NaN,NaN,2.39,0.75,0.78,33.41,NaN
3,2017-10-24 15:00:00,Saltillo,NaN,NaN,4.91,0.66,0.80,25.20,NaN
4,2017-10-24 16:00:00,Saltillo,NaN,NaN,2.80,0.20,0.78,18.40,NaN
...,...,...,...,...,...,...,...,...,...
197407,2017-10-24 07:00:00,Saltillo,NaN,NaN,29.62,24.92,1.04,31.90,NaN
197408,2017-10-24 08:00:00,Saltillo,NaN,NaN,27.69,17.76,1.14,49.27,NaN
197409,2017-10-24 09:00:00,Saltillo,NaN,NaN,17.20,9.27,0.97,45.39,NaN
197410,2017-10-24 10:00:00,Saltillo,NaN,NaN,10.04,6.24,0.91,33.80,NaN


#SPLIT DATA BY REGION

Uncomment the line for the region you want.

In [47]:
#pollution_clean = pollution_clean.loc[pollution_clean['region']=='Morelia']
#pollution_clean = pollution_clean.loc[pollution_clean['region']=='Saltillo']
pollution_clean = pollution_clean.loc[pollution_clean['region']=='Aguascalientes']

pollution_clean


,Time,region,O3,SO2,NO2,NO,CO,PM10,PM25
121184,2015-01-01 00:00:00,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121185,2015-01-01 01:00:00,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121186,2015-01-01 02:00:00,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121187,2015-01-01 03:00:00,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121188,2015-01-01 04:00:00,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
187167,2018-10-15 23:00:00,Aguascalientes,9.85,0.54,9.86,7.37,NaN,9.32,NaN
187168,2018-10-16 00:00:00,Aguascalientes,7.70,0.53,9.75,7.33,NaN,10.16,NaN
187169,2018-10-16 01:00:00,Aguascalientes,9.55,0.51,9.06,7.30,NaN,10.60,NaN
187170,2018-10-16 02:00:00,Aguascalientes,10.32,0.55,8.77,7.32,NaN,2.00,NaN


# SELECT SPECIFIC TIME PERIOD

insert speficfic start and end dates including time information following the format YYY-mm-dd HH:MM:SS

In [48]:
start_date ='2016-01-01 00:00:00'
end_date = '2016-12-31 23:00:00'

period = (pollution_clean['Time'] >= start_date) & (pollution_clean['Time'] <= end_date)

pollution_clean = pollution_clean.loc[period]

pollution_clean

,Time,region,O3,SO2,NO2,NO,CO,PM10,PM25
138136,2016-01-01 00:00:00,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138137,2016-01-01 01:00:00,Aguascalientes,2.76,0.25,14.88,NaN,2.98,120.75,NaN
138138,2016-01-01 02:00:00,Aguascalientes,3.31,0.25,15.33,NaN,2.86,124.76,NaN
138139,2016-01-01 03:00:00,Aguascalientes,2.24,0.25,15.16,39.00,2.26,101.97,NaN
138140,2016-01-01 04:00:00,Aguascalientes,2.47,NaN,14.50,33.44,2.18,85.84,81.12
...,...,...,...,...,...,...,...,...,...
155107,2016-12-31 19:00:00,Aguascalientes,43.81,NaN,2.13,0.20,1.55,34.22,29.95
155108,2016-12-31 20:00:00,Aguascalientes,41.40,NaN,3.13,0.20,1.57,32.28,25.12
155109,2016-12-31 21:00:00,Aguascalientes,44.20,NaN,3.14,0.20,1.57,32.93,23.13
155110,2016-12-31 22:00:00,Aguascalientes,40.76,NaN,5.60,1.29,1.62,28.74,28.52


# DALY AVERAGES AND DIURNAL DATA

In [49]:
'24h average'

pollution_clean = pollution_clean.assign(Date=pollution_clean.Time.dt.date)
#pollution_clean.loc[:,'Date'] = pollution_clean.Time.dt.date   # Create column with dates only

def calculate_daily_mean(group, is_circular=False):   # is_circular conditional for circular variables such as wind direction. To be declared on function custom_mean.
  valid_count = group.notna().sum()
  if valid_count >= 18:
    if is_circular:
      rad = np.deg2rad(group)
      mean_sin = np.mean(np.sin(rad))
      mean_cos = np.mean(np.cos(rad))
      mean_angle = np.arctan2(mean_sin, mean_cos)
      return np.rad2deg(mean_angle) % 360     # Ensure that mean_angle is between 0 an 360
    else:
      return group.mean().round(2)
  else:
    return np.nan

def custom_mean(group):
  return {
      'O3': calculate_daily_mean(group['O3']),
      'SO2': calculate_daily_mean(group['SO2']),
      'NO2': calculate_daily_mean(group['NO2']),
      'NO': calculate_daily_mean(group['NO']),
      'CO': calculate_daily_mean(group['CO']),
      'PM10': calculate_daily_mean(group['PM10']),
      'PM25': calculate_daily_mean(group['PM25']),
  }

daily_mean = pollution_clean.groupby(['region', 'Date']).apply(custom_mean, include_groups=False).apply(pd.Series)
daily_mean = daily_mean.reset_index()
display(daily_mean)
del pollution_clean['Date']

"hourly average (output: 24 lines) diurnal cycles taking the median for better representation"
# pollution_clean = pollution_clean.groupby(['region', pollution_clean['Time'].dt.hour]).median()
# pollution_clean



,region,Date,O3,SO2,NO2,NO,CO,PM10,PM25
0,Aguascalientes,2016-01-01,19.80,NaN,8.51,6.70,1.69,42.65,24.95
1,Aguascalientes,2016-01-02,NaN,NaN,14.69,12.50,2.21,27.00,11.90
2,Aguascalientes,2016-01-03,NaN,NaN,19.16,8.41,2.02,25.81,15.70
3,Aguascalientes,2016-01-04,35.89,NaN,7.48,5.98,2.14,39.52,7.73
4,Aguascalientes,2016-01-05,45.02,NaN,13.08,6.46,2.51,30.10,17.42
...,...,...,...,...,...,...,...,...,...
361,Aguascalientes,2016-12-27,26.27,NaN,14.01,2.08,1.59,38.06,24.76
362,Aguascalientes,2016-12-28,21.61,NaN,13.91,3.89,1.60,35.00,21.16
363,Aguascalientes,2016-12-29,17.82,NaN,12.85,4.46,1.59,31.41,17.06
364,Aguascalientes,2016-12-30,16.84,NaN,12.53,7.60,1.73,28.13,15.37


'hourly average (output: 24 lines) diurnal cycles taking the median for better representation'

#SAVING CLEANED DATASET AS CSV FILE

In [50]:
date_now = datetime.datetime.now() - datetime.timedelta(hours=6)
date_now = date_now.strftime('%Y-%m-%d %H:%M')

file_name = 'pollution_clean_' + str(date_now) + '.csv'

pollution_clean.to_csv('/content/drive/MyDrive/ATPAD_COLAB/OUTPUT_FILES/'+ file_name)

#daily_mean.to_csv('/content/drive/MyDrive/ATPAD_COLAB/OUTPUT_FILES/daily_mean_pollution_' + str(date_now) + '.csv')